In [31]:
import pandas as pd
import numpy as np

In [32]:
#directories
test_dir = '/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/Test_dataset.xlsx'
#loading data
test_data =pd.read_excel(test_dir,'Foreign_Visitors_TS')
#changing names

In [33]:
test_data=test_data.rename(columns={'Time Series Data':'2020/04/01','Unnamed: 2':'2020/05/01','Unnamed: 3':'2020/06/01','Unnamed: 4':'2020/07/01','Unnamed: 5':'2020/08/01'}) 


In [34]:
#dropping first row(which is nothing but date and time)
test_data=test_data.drop([0],axis=0)
#show top 5 rows of training dataframe
test_data.head()

,City,2020/04/01,2020/05/01,2020/06/01,2020/07/01,2020/08/01
1,Tuensang,1614,1946,2372,2500,2769
2,Lakshmeshwar,369179,445559,541045,572860,636502
3,Zira,140581,169665,206030,218138,242367
4,Yawal,2557179,3086249,3747597,3968032,4408916
5,Thana Bhawan,1800363,2172850,2638469,2793662,3104060


In [35]:
test_data.dtypes

City          object
2020/04/01    object
2020/05/01    object
2020/06/01    object
2020/07/01    object
2020/08/01    object
dtype: object

In [36]:
test_data['2020/04/01'] = test_data['2020/04/01'].astype(float)
test_data['2020/05/01'] = test_data['2020/05/01'].astype(float)
test_data['2020/06/01'] = test_data['2020/06/01'].astype(float)
test_data['2020/07/01'] = test_data['2020/07/01'].astype(float)
test_data['2020/08/01'] = test_data['2020/08/01'].astype(float)

In [37]:
City = test_data["City"].values

In [38]:
test_data.drop(['City'],axis=1,inplace=True)

In [39]:
test_data=test_data.transpose()

In [40]:
for i in range(1,502):
    test_data.replace(np.NaN,{i:test_data[i].mean()},inplace=True)

In [41]:
test_data=test_data.transpose()

In [42]:
test_data

,2020/04/01,2020/05/01,2020/06/01,2020/07/01,2020/08/01
1,1614.0,1946.0,2372.0,2500.0,2769.0
2,369179.0,445559.0,541045.0,572860.0,636502.0
3,140581.0,169665.0,206030.0,218138.0,242367.0
4,2557179.0,3086249.0,3747597.0,3968032.0,4408916.0
5,1800363.0,2172850.0,2638469.0,2793662.0,3104060.0
...,...,...,...,...,...
497,61420.0,74125.0,90018.0,95302.0,105882.0
498,61420.0,74125.0,90018.0,95302.0,105882.0
499,61420.0,74125.0,90018.0,95302.0,105882.0
500,61420.0,74125.0,90018.0,95302.0,105882.0


In [43]:

def split_dataset(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix == len(sequence):
            break
        seq_x = sequence[i:end_ix+1]
        X.append(seq_x)
    return pd.DataFrame(X)

In [44]:
def get_data(data,n_steps):
    database = pd.DataFrame()
    for i in range(len(data)):
        arr = split_dataset(data.values[i],n_steps)
        database = database.append(arr, ignore_index=True)
    print("data:", data.shape)
    print("database:", database.shape)
    return database

In [46]:
test_data.values[0]

array([1614., 1946., 2372., 2500., 2769.])

In [45]:
n_steps = 1
train_database= get_data(test_data,n_steps)
Y_train = train_database[n_steps].values
train_database = train_database.drop(n_steps, axis=1)
X_train = train_database.values
print(X_train.shape)
print(Y_train.shape)

data: (501, 5)
database: (2004, 2)
(2004, 1)
(2004,)


In [48]:
np.savez("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/time_data.npz", X_train=X_train, Y_train=Y_train, City= City, n_steps=np.array([n_steps]))

In [52]:
import numpy as np
data = np.load("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/time_data.npz")
X_train = data["X_train"]
Y_train = data["Y_train"]

n_steps = data["n_steps"][0]

In [53]:
print(n_steps)

1


In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [69]:
model = Sequential()
model.add(Input((n_steps,1)))
model.add(LSTM(units=100, activation="relu",return_sequences=True))
model.add(LSTM(units=55, activation="relu",return_sequences=True))
model.add(Dense(1))
model.compile(optimizer="adam", loss="msle")

In [70]:
X = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [71]:
Y = Y_train.reshape(-1,1)

In [72]:

print("X:", X.shape)
print("Y:", Y.shape)

X: (2004, 1, 1)
Y: (2004, 1)


In [73]:
model.fit(X,Y,epochs=30, validation_split=0.2)

Epoch 1/30
51/51 [==============================] - 1s 10ms/step - loss: 10.9213 - val_loss: 6.5473
Epoch 2/30
51/51 [==============================] - 0s 4ms/step - loss: 5.0499 - val_loss: 3.7814
Epoch 3/30
51/51 [==============================] - 0s 4ms/step - loss: 3.0307 - val_loss: 2.3552
Epoch 4/30
51/51 [==============================] - 0s 4ms/step - loss: 1.9240 - val_loss: 1.5221
Epoch 5/30
51/51 [==============================] - 0s 4ms/step - loss: 1.1805 - val_loss: 0.8194
Epoch 6/30
51/51 [==============================] - 0s 4ms/step - loss: 0.6060 - val_loss: 0.4191
Epoch 7/30
51/51 [==============================] - 0s 4ms/step - loss: 0.3151 - val_loss: 0.2202
Epoch 8/30
51/51 [==============================] - 0s 4ms/step - loss: 0.1678 - val_loss: 0.1173
Epoch 9/30
51/51 [==============================] - 0s 4ms/step - loss: 0.0906 - val_loss: 0.0630
Epoch 10/30
51/51 [==============================] - 0s 4ms/step - loss: 0.0496 - val_loss: 0.0341
Epoch 11/30
51/51

In [74]:
model.save("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/Time_model_real.h5")

In [75]:

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1, 100)            40800     
_________________________________________________________________
lstm_8 (LSTM)                (None, 1, 55)             34320     
_________________________________________________________________
dense_5 (Dense)              (None, 1, 1)              56        
Total params: 75,176
Trainable params: 75,176
Non-trainable params: 0
_________________________________________________________________


In [77]:
from tensorflow.keras.models import load_model
time_model = load_model("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/Time_model_real.h5")

In [80]:
test_dataset = pd.read_excel("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/Test_dataset.xlsx")
test_fv = test_dataset["Foreign Visitors"]

In [81]:
input_arr = test_fv.values.reshape(-1,1,1)
print(input_arr.shape)

(501, 1, 1)


In [82]:
n=1
for i in range(n):
    input_arr = time_model.predict(input_arr)
    input_arr = input_arr.reshape(-1,1,1)


In [83]:
input_arr = input_arr.flatten()

In [84]:
test_dataset["Foreign Visitors"] = input_arr.astype("int64")

In [86]:
test_dataset.to_excel("/content/drive/My Drive/fwdfliprhackathon6_0machinelearningtask/October_test_dataset.xlsx")

In [87]:
input_arr

array([3.13487207e+03, 7.20371562e+05, 2.74303438e+05, 4.98985600e+06,
       3.51306675e+06, 7.20371562e+05, 4.98985600e+06, 1.42691672e+05,
       7.20371562e+05, 1.66970575e+06, 1.68576438e+06, 4.98985600e+06,
       6.45774561e+03, 1.89894172e+05, 4.98985600e+06, 7.20371562e+05,
       7.20371562e+05, 4.76887031e+05, 4.76887031e+05, 2.69195781e+05,
       7.20371562e+05, 1.66970575e+06, 6.62862188e+04, 4.98985600e+06,
       1.66970575e+06, 7.20371562e+05, 5.30198700e+06, 3.51306675e+06,
       3.13487207e+03, 3.22523406e+05, 7.20371562e+05, 1.66970575e+06,
                  nan, 4.98985600e+06, 4.98985600e+06, 2.74303438e+05,
       3.51306675e+06, 1.66970575e+06, 4.76887031e+05, 1.66970575e+06,
       5.30198700e+06, 4.98985600e+06, 4.59619750e+05, 3.22523406e+05,
       1.19834531e+05, 4.76887031e+05, 1.04545388e+06, 1.04545388e+06,
       4.76887031e+05, 4.98985600e+06, 1.89894172e+05, 1.89894172e+05,
       1.10627725e+06, 7.57964375e+04, 3.22523406e+05, 2.69195781e+05,
      